In [68]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import numpy as np

**Achtung**, weil wir einmal neu einsetzen mussten, müssen wird das Ergebnis noch auf Duplikate prüfen

Ansonsten Vorgehen: 
- mit os fileliste rausziehen
- loop bauen mit list comprehension
- df bauen

# Problem:
- Wir haben gemerkt, dass as Zeugs manchmal verrutscht ist. Wir haben versucht, das in der Funktion mit Try und Except in den Griff zu bekommen (indem wir auf andere Zeile zugreifen). Aber das funktioniert nicht. 
- Entweder (vielversprechender): Wir machen die Definitionen ganz neu, inedem wir direkt mit Regex ins soup.find_all("td")[0] einsteigen, wo alle Infos drin sind.
- oder wir versuchen fixe Muster zu finden...

In [2]:
allefiles = os.listdir("simap1_2019")

In [3]:
file = open("simap1_2019/zuschlag632.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

In [4]:
# Hier ist der Name des Auftraggebers drin:
datablock = soup.find_all("td")[0]

In [5]:
datablock = str(soup.find_all("td")[0])

datablock = datablock.replace("\n", " ")
datablock = datablock.replace("\xa0", " ")
datablock = datablock.replace("\n", "")
datablock = datablock.replace("\t", "")

datablock

'<td><h1>Zuschlag</h1><dl><dd><span class="label">Publikationsdatum Simap:</span> 01.06.2018<br/></dd></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span> Stadt Winterthur Amt für Städtebau<br/><span class="label">Beschaffungsstelle/Organisator:</span> Stadt Winterthur<br/>Amt für Städtebau, Pionierstrasse 7,  8403  Winterthur,  Schweiz,    Telefon:    052 267 54 62,  E-Mail:  <a href="mailto:staedtebau@win.ch">staedtebau@win.ch</a><br/></dd></dl><h3>1.2 Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3 Verfahrensart </h3><dl><dd> Freihändiges Verfahren <br/></dd></dl><h3>1.4 Auftragsart </h3><dl><dd> Bauauftrag <br/></dd></dl><h3>1.5 Gemäss GATT/WTO-Abkommen, resp. Staatsvertrag </h3><dl><dd> Nein <br/></dd></dl><h2>2. Beschaffungsobjekt</h2><h3>2.1 Projekttitel der Beschaffung</h3><dl><dd>Theater Winterthur; zusätzliche Arbeiten an der Audio- und Videotechnik; Revision S

In [6]:
regex1 = r"(?<=Bedarfsstelle\/Vergabestelle:<\/span> ).*(?=<br\/><span class=\"label\">Beschaffungsstelle\/Organisator:)"
name = re.findall(regex1, datablock)[0]
name

'Stadt Winterthur Amt für Städtebau'

In [7]:
# Ausnahme für Französisch:
regex1 = r"(?<=adjudicatrice:<\/span> ).*(?=<br\/><span class=\"label\">Service organisateur)"
name = re.findall(regex1, datablock)[0]
name

IndexError: list index out of range

In [9]:
#test im Loop:
allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")

    datablock
        
    regex1 = r"(?<=Bedarfsstelle\/Vergabestelle:<\/span> ).*(?=<br\/><span class=\"label\">Beschaffungsstelle\/Organisator:)"
    name = re.findall(regex1, datablock)[0]
    
    mini_dict = {"Name" : name}
    return mini_dict


for file in allefiles:
    data = open("simap1_2019/"+str(file), "r")
    text = data.read()
    soup = BeautifulSoup(text, "html.parser")
    
    list_df.append(my_function(soup))
    print(file)

zuschlag768.htm
zuschlag2393.htm
zuschlag1842.htm
zuschlag1856.htm
zuschlag3099.htm
zuschlag2387.htm
zuschlag754.htm
zuschlag740.htm
zuschlag998.htm
zuschlag2436.htm
zuschlag973.htm
zuschlag1881.htm
zuschlag1659.htm
zuschlag2350.htm
zuschlag2344.htm
zuschlag1895.htm
zuschlag967.htm
zuschlag2422.htm
zuschlag1103.htm
zuschlag797.htm
zuschlag3072.htm
zuschlag1665.htm
zuschlag1671.htm
zuschlag2378.htm
zuschlag3066.htm
zuschlag1117.htm
zuschlag783.htm
zuschlag1498.htm
zuschlag2191.htm
zuschlag58.htm
zuschlag2185.htm
zuschlag218.htm
zuschlag230.htm
zuschlag556.htm
zuschlag70.htm
zuschlag2813.htm
zuschlag2807.htm
zuschlag64.htm
zuschlag542.htm
zuschlag224.htm
zuschlag2152.htm
zuschlag2634.htm
zuschlag2620.htm
zuschlag1329.htm
zuschlag2146.htm
zuschlag1467.htm
zuschlag2608.htm
zuschlag595.htm
zuschlag1301.htm
zuschlag581.htm
zuschlag1315.htm
zuschlag1473.htm
zuschlag378.htm
zuschlag2783.htm
zuschlag2797.htm
zuschlag344.htm
zuschlag422.htm
zuschlag2967.htm
zuschlag2973.htm
zuschlag436.htm
zusch

IndexError: list index out of range

### Fazit: Bis file 1063 geht das jetzt beim Namen, dann kommt der Fehler – also bei File zuschlag3113
- Ergebis: Weil französisch!!!
> Also machen wir dafür eine Asunahme: "Service demandeur/Entité adjudicatrice:

Jetzt kommen wir bis zu File 632 - hmm: da hats in der Fileliste ein seltsames Buggy-File "DS-STore" drin > ich könnte einen Check einbauen: Filename muss "zuschlag" enthalten

### Heureka, jetzt geht's – also das gleiche Prozedere mit den anderen Werten wiederholen!

In [8]:
#test im Loop:
allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")

    datablock
    
    try: 
        regex1 = r"(?<=Bedarfsstelle\/Vergabestelle:<\/span> ).*(?=<br\/><span class=\"label\">Beschaffungsstelle\/Organisator:)"
        name = re.findall(regex1, datablock)[0]
    except: 
        regex1 = r"(?<=adjudicatrice:<\/span> ).*(?=<br\/><span class=\"label\">Service organisateur)"
        name = re.findall(regex1, datablock)[0]

    mini_dict = {"Name" : name}
    return mini_dict


for file in allefiles:
    if "zuschlag" in file: 
        data = open("simap1_2019/"+str(file), "r")
        text = data.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print(file)
        list_df.append(my_function(soup))

zuschlag768.htm
zuschlag2393.htm
zuschlag1842.htm
zuschlag1856.htm
zuschlag3099.htm
zuschlag2387.htm
zuschlag754.htm
zuschlag740.htm
zuschlag998.htm
zuschlag2436.htm
zuschlag973.htm
zuschlag1881.htm
zuschlag1659.htm
zuschlag2350.htm
zuschlag2344.htm
zuschlag1895.htm
zuschlag967.htm
zuschlag2422.htm
zuschlag1103.htm
zuschlag797.htm
zuschlag3072.htm
zuschlag1665.htm
zuschlag1671.htm
zuschlag2378.htm
zuschlag3066.htm
zuschlag1117.htm
zuschlag783.htm
zuschlag1498.htm
zuschlag2191.htm
zuschlag58.htm
zuschlag2185.htm
zuschlag218.htm
zuschlag230.htm
zuschlag556.htm
zuschlag70.htm
zuschlag2813.htm
zuschlag2807.htm
zuschlag64.htm
zuschlag542.htm
zuschlag224.htm
zuschlag2152.htm
zuschlag2634.htm
zuschlag2620.htm
zuschlag1329.htm
zuschlag2146.htm
zuschlag1467.htm
zuschlag2608.htm
zuschlag595.htm
zuschlag1301.htm
zuschlag581.htm
zuschlag1315.htm
zuschlag1473.htm
zuschlag378.htm
zuschlag2783.htm
zuschlag2797.htm
zuschlag344.htm
zuschlag422.htm
zuschlag2967.htm
zuschlag2973.htm
zuschlag436.htm
zusch

zuschlag829.htm
zuschlag1065.htm
zuschlag1071.htm
zuschlag2578.htm
zuschlag3100.htm
zuschlag1717.htm
zuschlag183.htm
zuschlag3128.htm
zuschlag2236.htm
zuschlag815.htm
zuschlag1059.htm
zuschlag2550.htm
zuschlag2544.htm
zuschlag801.htm
zuschlag2222.htm
zuschlag800.htm
zuschlag2545.htm
zuschlag2223.htm
zuschlag2237.htm
zuschlag3129.htm
zuschlag2551.htm
zuschlag1058.htm
zuschlag814.htm
zuschlag2579.htm
zuschlag1070.htm
zuschlag182.htm
zuschlag1716.htm
zuschlag3101.htm
zuschlag3115.htm
zuschlag196.htm
zuschlag1702.htm
zuschlag1064.htm
zuschlag828.htm
zuschlag2586.htm
zuschlag1931.htm
zuschlag169.htm
zuschlag1925.htm
zuschlag2592.htm
zuschlag627.htm
zuschlag141.htm
zuschlag155.htm
zuschlag1919.htm
zuschlag633.htm
zuschlag1528.htm
zuschlag2021.htm
zuschlag2747.htm
zuschlag2753.htm
zuschlag2035.htm
zuschlag1514.htm
zuschlag380.htm
zuschlag1272.htm
zuschlag1266.htm
zuschlag1500.htm
zuschlag394.htm
zuschlag2009.htm
zuschlag419.htm
zuschlag2784.htm
zuschlag2948.htm
zuschlag2790.htm
zuschlag1299.h

zuschlag3010.htm
zuschlag1607.htm
zuschlag1161.htm
zuschlag2468.htm
zuschlag1149.htm
zuschlag905.htm
zuschlag2440.htm
zuschlag3038.htm
zuschlag2326.htm
zuschlag2332.htm
zuschlag2454.htm
zuschlag911.htm
zuschlag722.htm
zuschlag1808.htm
zuschlag736.htm
zuschlag2483.htm
zuschlag1834.htm
zuschlag1820.htm
zuschlag2497.htm
zuschlag1836.htm
zuschlag1188.htm
zuschlag2481.htm
zuschlag2495.htm
zuschlag708.htm
zuschlag1822.htm
zuschlag720.htm
zuschlag734.htm
zuschlag2324.htm
zuschlag2442.htm
zuschlag907.htm
zuschlag913.htm
zuschlag2456.htm
zuschlag2330.htm
zuschlag1639.htm
zuschlag2318.htm
zuschlag3006.htm
zuschlag1611.htm
zuschlag1177.htm
zuschlag1163.htm
zuschlag1605.htm
zuschlag3012.htm
zuschlag2683.htm
zuschlag38.htm
zuschlag278.htm
zuschlag2697.htm
zuschlag522.htm
zuschlag2867.htm
zuschlag244.htm
zuschlag250.htm
zuschlag2873.htm
zuschlag10.htm
zuschlag536.htm
zuschlag1349.htm
zuschlag0.htm
zuschlag2640.htm
zuschlag2898.htm
zuschlag2126.htm
zuschlag2132.htm
zuschlag2654.htm
zuschlag1375.htm
z

zuschlag2530.htm
zuschlag875.htm
zuschlag1039.htm
zuschlag2257.htm
zuschlag1986.htm
zuschlag1038.htm
zuschlag874.htm
zuschlag2531.htm
zuschlag2525.htm
zuschlag860.htm
zuschlag1992.htm
zuschlag2243.htm
zuschlag1762.htm
zuschlag1004.htm
zuschlag848.htm
zuschlag690.htm
zuschlag1010.htm
zuschlag684.htm
zuschlag2519.htm
zuschlag1776.htm
zuschlag109.htm
zuschlag1945.htm
zuschlag2294.htm
zuschlag2280.htm
zuschlag1789.htm
zuschlag1951.htm
zuschlag135.htm
zuschlag1979.htm
zuschlag653.htm
zuschlag647.htm
zuschlag121.htm
zuschlag2733.htm
zuschlag2055.htm
zuschlag2041.htm
zuschlag1548.htm
zuschlag2727.htm
zuschlag492.htm
zuschlag1206.htm
zuschlag2069.htm
zuschlag1560.htm
zuschlag1574.htm
zuschlag486.htm
zuschlag1212.htm
zuschlag2928.htm
zuschlag2096.htm
zuschlag2082.htm
zuschlag479.htm
zuschlag2914.htm
zuschlag451.htm
zuschlag337.htm
zuschlag323.htm
zuschlag445.htm
zuschlag2900.htm
zuschlag2647.htm
zuschlag7.htm
zuschlag1428.htm
zuschlag2121.htm
zuschlag2135.htm
zuschlag2653.htm
zuschlag1372.htm
z

zuschlag2472.htm
zuschlag937.htm
zuschlag260.htm
zuschlag2843.htm
zuschlag1392.htm
zuschlag506.htm
zuschlag20.htm
zuschlag1386.htm
zuschlag34.htm
zuschlag512.htm
zuschlag2857.htm
zuschlag274.htm
zuschlag248.htm
zuschlag1437.htm
zuschlag1351.htm
zuschlag2658.htm
zuschlag2880.htm
zuschlag2894.htm
zuschlag1345.htm
zuschlag1423.htm
zuschlag2102.htm
zuschlag2664.htm
zuschlag1379.htm
zuschlag2670.htm
zuschlag2116.htm
zuschlag1580.htm
zuschlag314.htm
zuschlag2089.htm
zuschlag2937.htm
zuschlag472.htm
zuschlag466.htm
zuschlag2923.htm
zuschlag1594.htm
zuschlag300.htm
zuschlag328.htm
zuschlag1543.htm
zuschlag1225.htm
zuschlag2738.htm
zuschlag1231.htm
zuschlag1557.htm
zuschlag2076.htm
zuschlag2710.htm
zuschlag1219.htm
zuschlag499.htm
zuschlag2704.htm
zuschlag2062.htm
zuschlag670.htm
zuschlag116.htm
zuschlag1782.htm
zuschlag102.htm
zuschlag1796.htm
zuschlag664.htm
zuschlag894.htm
zuschlag1966.htm
zuschlag1972.htm
zuschlag880.htm
zuschlag658.htm
zuschlag1027.htm
zuschlag1741.htm
zuschlag1999.htm
zus

zuschlag402.htm
zuschlag2947.htm
zuschlag358.htm
zuschlag1335.htm
zuschlag87.htm
zuschlag1453.htm
zuschlag1447.htm
zuschlag2628.htm
zuschlag1321.htm
zuschlag93.htm
zuschlag2600.htm
zuschlag1309.htm
zuschlag2166.htm
zuschlag2172.htm
zuschlag589.htm
zuschlag2614.htm
zuschlag2827.htm
zuschlag562.htm
zuschlag44.htm
zuschlag1490.htm
zuschlag204.htm
zuschlag2199.htm
zuschlag1484.htm
zuschlag210.htm
zuschlag50.htm
zuschlag576.htm
zuschlag2833.htm
zuschlag78.htm
zuschlag238.htm
zuschlag1651.htm
zuschlag1889.htm
zuschlag3046.htm
zuschlag2358.htm
zuschlag1137.htm
zuschlag1123.htm
zuschlag3052.htm
zuschlag1645.htm
zuschlag2364.htm
zuschlag947.htm
zuschlag2402.htm
zuschlag2416.htm
zuschlag953.htm
zuschlag1679.htm
zuschlag2370.htm
zuschlag3085.htm
zuschlag1692.htm
zuschlag760.htm
zuschlag774.htm
zuschlag1686.htm
zuschlag3091.htm
zuschlag1876.htm
zuschlag984.htm
zuschlag990.htm
zuschlag748.htm
zuschlag1862.htm
zuschlag986.htm
zuschlag1874.htm
zuschlag1860.htm
zuschlag992.htm
zuschlag762.htm
zuschlag

zuschlag214.htm
zuschlag1480.htm
zuschlag2189.htm
zuschlag200.htm
zuschlag1494.htm
zuschlag40.htm
zuschlag566.htm
zuschlag2823.htm
zuschlag68.htm
zuschlag228.htm
zuschlag97.htm
zuschlag1325.htm
zuschlag1443.htm
zuschlag1457.htm
zuschlag2638.htm
zuschlag83.htm
zuschlag1331.htm
zuschlag2610.htm
zuschlag1319.htm
zuschlag2176.htm
zuschlag2162.htm
zuschlag599.htm
zuschlag2604.htm
zuschlag2943.htm
zuschlag406.htm
zuschlag1292.htm
zuschlag360.htm
zuschlag374.htm
zuschlag412.htm
zuschlag1286.htm
zuschlag2957.htm
zuschlag348.htm
zuschlag1251.htm
zuschlag2980.htm
zuschlag2758.htm
zuschlag1537.htm
zuschlag1523.htm
zuschlag2994.htm
zuschlag1245.htm
zuschlag2764.htm
zuschlag2002.htm
zuschlag2016.htm
zuschlag1279.htm
zuschlag2770.htm
zuschlag162.htm
zuschlag1090.htm
zuschlag604.htm
zuschlag2599.htm
zuschlag1084.htm
zuschlag610.htm
zuschlag176.htm
zuschlag1912.htm
zuschlag638.htm
zuschlag1906.htm
zuschlag1735.htm
zuschlag3122.htm
zuschlag1053.htm
zuschlag1047.htm
zuschlag3136.htm
zuschlag2228.htm
zus

In [22]:
df = pd.DataFrame(list_df)
df["Name"].value_counts().head(20)

Stadt Zürich                                              989
Stadt Zürich, Amt für Hochbauten                          282
Gemeinde Meilen, Liegenschaftsabteilung                   150
Stadt Winterthur Amt für Städtebau                        100
ewz Elektrizitätswerk der Stadt Zürich                     86
Stadt Zürich, Dienstabteilung Verkehr                      74
Stadt Uster, Abteilung Gesundheit                          63
Stadt Winterthur Departement Bau                           52
Zürichsee Schifffahrtsgesellschaft                         50
Politische Gemeinde Erlenbach Liegenschaftenkommission     42
Gemeinde Maur                                              41
Einfache Gesellschaft Kunsthaus Erweiterung                40
Kongresshaus-Stiftung Zürich                               39
Stadt Opfikon Glattbrugg                                   35
Grün Stadt Zürich                                          34
Gemeinde Zollikon                                          32
Stadt Op

### 2.1 Jetzt probieren wir das gleiche mit den anderen Werten:
- Anbieter, Adresse
- Projekt, Projekttyp
- Preis etc.

In [48]:
datablock = str(soup.find_all("td")[0])
datablock

'<td><h1>Zuschlag</h1><dl><dd></dd><dd><b>Publikationsdatum Kantonales Amtsblatt</b></dd>\n\t\t\t\t\t\xa0\n\t\t\t\t\tZH\xa0\t\t\t\t\t\n\t\t\t\t\t\t01.03.2013<br/><span class="label">Publikationsdatum Simap:</span>\xa001.03.2013<br/></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span>\xa0Politische Gemeinde Rüti<br/><span class="label">Beschaffungsstelle/Organisator:</span>\xa0Bauamt Rüti,\xa0\n\t\t  zu Hdn. von \n\t\t  Jan Schaufelberger,\xa0Breitenhofstrasse 30,\xa0\n\t\t8630\xa0\n\t\tRüti,\xa0\n\t\tSchweiz,\xa0\t\t\t\n\t\t  Telefon:\xa0\n\t\t  055 251 32 10,\xa0\t\t\t\n\t\t  Fax:\xa0\n\t\t  055 251 32 13,\xa0\t\t\t\t\t\t\n\t\t\tE-Mail:\xa0\t\t\t\t\n\t\t\t\t<a href="mailto:bauamt@rueti.ch">bauamt@rueti.ch</a>,\xa0\t\t\t\n\t\t  URL\n\t\t  www.rueti.ch<br/></dd></dl><h3>1.2\xa0Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3\n\t\t\tVerfahrensart\n\t\t</h3><dl><dd>\n\t\t\t\

In [49]:
datablock = str(soup.find_all("td")[0])

datablock = datablock.replace("\n", " ")
datablock = datablock.replace("\xa0", " ")
datablock = datablock.replace("\n", "")
datablock = datablock.replace("\t", "")
datablock = datablock.replace("\'", "")

datablock

'<td><h1>Zuschlag</h1><dl><dd></dd><dd><b>Publikationsdatum Kantonales Amtsblatt</b></dd>   ZH  01.03.2013<br/><span class="label">Publikationsdatum Simap:</span> 01.03.2013<br/></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span> Politische Gemeinde Rüti<br/><span class="label">Beschaffungsstelle/Organisator:</span> Bauamt Rüti,    zu Hdn. von    Jan Schaufelberger, Breitenhofstrasse 30,  8630  Rüti,  Schweiz,    Telefon:    055 251 32 10,    Fax:    055 251 32 13,  E-Mail:  <a href="mailto:bauamt@rueti.ch">bauamt@rueti.ch</a>,    URL   www.rueti.ch<br/></dd></dl><h3>1.2 Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3 Verfahrensart </h3><dl><dd> Offenes Verfahren <br/></dd></dl><h3>1.4 Auftragsart </h3><dl><dd> Bauauftrag <br/></dd></dl><h3>1.5 Gemäss GATT/WTO-Abkommen, resp. Staatsvertrag </h3><dl><dd> Nein <br/></dd></dl><h2>2. Beschaffungsobjekt</h2><h3>2.1 Projektti

In [27]:
regex2 = r"(?<=Name:<\/span> ).*(?=<br\/><span class="label"\>Preis)"
name = re.findall(regex1, datablock)[0]
name

SyntaxError: invalid syntax (<ipython-input-27-613222ab5eed>, line 1)

In [37]:
#test2 im Loop (Zuschlag):
allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")

    datablock
    
    try: 
        regex2 = r"(?<=Name:<\/span> ).*(?=<br\/><span class=\"label\"\>Preis)"
        zuschlag = re.findall(regex2, datablock)[0]
    except:
        try: 
            regex2 = r"(?<=Nom:<\/span> ).*(?=<br\/><span class=\"label\"\>Prix)"
            zuschlag = re.findall(regex2, datablock)[0]
        except: 
            regex2 = r"(?<=Nom:<\/span> ).*(?=<br\/><span class=\"label\"\>Fourchette)"
            zuschlag = re.findall(regex2, datablock)[0]

    mini_dict = {"Zuschlag" : zuschlag}
    return mini_dict


for file in allefiles:
    if "zuschlag" in file: 
        data = open("simap1_2019/"+str(file), "r")
        text = data.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print(file)
        list_df.append(my_function(soup))

zuschlag768.htm
zuschlag2393.htm
zuschlag1842.htm
zuschlag1856.htm
zuschlag3099.htm
zuschlag2387.htm
zuschlag754.htm
zuschlag740.htm
zuschlag998.htm
zuschlag2436.htm
zuschlag973.htm
zuschlag1881.htm
zuschlag1659.htm
zuschlag2350.htm
zuschlag2344.htm
zuschlag1895.htm
zuschlag967.htm
zuschlag2422.htm
zuschlag1103.htm
zuschlag797.htm
zuschlag3072.htm
zuschlag1665.htm
zuschlag1671.htm
zuschlag2378.htm
zuschlag3066.htm
zuschlag1117.htm
zuschlag783.htm
zuschlag1498.htm
zuschlag2191.htm
zuschlag58.htm
zuschlag2185.htm
zuschlag218.htm
zuschlag230.htm
zuschlag556.htm
zuschlag70.htm
zuschlag2813.htm
zuschlag2807.htm
zuschlag64.htm
zuschlag542.htm
zuschlag224.htm
zuschlag2152.htm
zuschlag2634.htm
zuschlag2620.htm
zuschlag1329.htm
zuschlag2146.htm
zuschlag1467.htm
zuschlag2608.htm
zuschlag595.htm
zuschlag1301.htm
zuschlag581.htm
zuschlag1315.htm
zuschlag1473.htm
zuschlag378.htm
zuschlag2783.htm
zuschlag2797.htm
zuschlag344.htm
zuschlag422.htm
zuschlag2967.htm
zuschlag2973.htm
zuschlag436.htm
zusch

zuschlag1930.htm
zuschlag1703.htm
zuschlag197.htm
zuschlag3114.htm
zuschlag829.htm
zuschlag1065.htm
zuschlag1071.htm
zuschlag2578.htm
zuschlag3100.htm
zuschlag1717.htm
zuschlag183.htm
zuschlag3128.htm
zuschlag2236.htm
zuschlag815.htm
zuschlag1059.htm
zuschlag2550.htm
zuschlag2544.htm
zuschlag801.htm
zuschlag2222.htm
zuschlag800.htm
zuschlag2545.htm
zuschlag2223.htm
zuschlag2237.htm
zuschlag3129.htm
zuschlag2551.htm
zuschlag1058.htm
zuschlag814.htm
zuschlag2579.htm
zuschlag1070.htm
zuschlag182.htm
zuschlag1716.htm
zuschlag3101.htm
zuschlag3115.htm
zuschlag196.htm
zuschlag1702.htm
zuschlag1064.htm
zuschlag828.htm
zuschlag2586.htm
zuschlag1931.htm
zuschlag169.htm
zuschlag1925.htm
zuschlag2592.htm
zuschlag627.htm
zuschlag141.htm
zuschlag155.htm
zuschlag1919.htm
zuschlag633.htm
zuschlag1528.htm
zuschlag2021.htm
zuschlag2747.htm
zuschlag2753.htm
zuschlag2035.htm
zuschlag1514.htm
zuschlag380.htm
zuschlag1272.htm
zuschlag1266.htm
zuschlag1500.htm
zuschlag394.htm
zuschlag2009.htm
zuschlag419.ht

zuschlag939.htm
zuschlag1613.htm
zuschlag3004.htm
zuschlag3010.htm
zuschlag1607.htm
zuschlag1161.htm
zuschlag2468.htm
zuschlag1149.htm
zuschlag905.htm
zuschlag2440.htm
zuschlag3038.htm
zuschlag2326.htm
zuschlag2332.htm
zuschlag2454.htm
zuschlag911.htm
zuschlag722.htm
zuschlag1808.htm
zuschlag736.htm
zuschlag2483.htm
zuschlag1834.htm
zuschlag1820.htm
zuschlag2497.htm
zuschlag1836.htm
zuschlag1188.htm
zuschlag2481.htm
zuschlag2495.htm
zuschlag708.htm
zuschlag1822.htm
zuschlag720.htm
zuschlag734.htm
zuschlag2324.htm
zuschlag2442.htm
zuschlag907.htm
zuschlag913.htm
zuschlag2456.htm
zuschlag2330.htm
zuschlag1639.htm
zuschlag2318.htm
zuschlag3006.htm
zuschlag1611.htm
zuschlag1177.htm
zuschlag1163.htm
zuschlag1605.htm
zuschlag3012.htm
zuschlag2683.htm
zuschlag38.htm
zuschlag278.htm
zuschlag2697.htm
zuschlag522.htm
zuschlag2867.htm
zuschlag244.htm
zuschlag250.htm
zuschlag2873.htm
zuschlag10.htm
zuschlag536.htm
zuschlag1349.htm
zuschlag0.htm
zuschlag2640.htm
zuschlag2898.htm
zuschlag2126.htm
zu

zuschlag1993.htm
zuschlag1987.htm
zuschlag2256.htm
zuschlag2530.htm
zuschlag875.htm
zuschlag1039.htm
zuschlag2257.htm
zuschlag1986.htm
zuschlag1038.htm
zuschlag874.htm
zuschlag2531.htm
zuschlag2525.htm
zuschlag860.htm
zuschlag1992.htm
zuschlag2243.htm
zuschlag1762.htm
zuschlag1004.htm
zuschlag848.htm
zuschlag690.htm
zuschlag1010.htm
zuschlag684.htm
zuschlag2519.htm
zuschlag1776.htm
zuschlag109.htm
zuschlag1945.htm
zuschlag2294.htm
zuschlag2280.htm
zuschlag1789.htm
zuschlag1951.htm
zuschlag135.htm
zuschlag1979.htm
zuschlag653.htm
zuschlag647.htm
zuschlag121.htm
zuschlag2733.htm
zuschlag2055.htm
zuschlag2041.htm
zuschlag1548.htm
zuschlag2727.htm
zuschlag492.htm
zuschlag1206.htm
zuschlag2069.htm
zuschlag1560.htm
zuschlag1574.htm
zuschlag486.htm
zuschlag1212.htm
zuschlag2928.htm
zuschlag2096.htm
zuschlag2082.htm
zuschlag479.htm
zuschlag2914.htm
zuschlag451.htm
zuschlag337.htm
zuschlag323.htm
zuschlag445.htm
zuschlag2900.htm
zuschlag2647.htm
zuschlag7.htm
zuschlag1428.htm
zuschlag2121.htm
z

zuschlag1621.htm
zuschlag1147.htm
zuschlag923.htm
zuschlag2466.htm
zuschlag2300.htm
zuschlag1609.htm
zuschlag2314.htm
zuschlag2472.htm
zuschlag937.htm
zuschlag260.htm
zuschlag2843.htm
zuschlag1392.htm
zuschlag506.htm
zuschlag20.htm
zuschlag1386.htm
zuschlag34.htm
zuschlag512.htm
zuschlag2857.htm
zuschlag274.htm
zuschlag248.htm
zuschlag1437.htm
zuschlag1351.htm
zuschlag2658.htm
zuschlag2880.htm
zuschlag2894.htm
zuschlag1345.htm
zuschlag1423.htm
zuschlag2102.htm
zuschlag2664.htm
zuschlag1379.htm
zuschlag2670.htm
zuschlag2116.htm
zuschlag1580.htm
zuschlag314.htm
zuschlag2089.htm
zuschlag2937.htm
zuschlag472.htm
zuschlag466.htm
zuschlag2923.htm
zuschlag1594.htm
zuschlag300.htm
zuschlag328.htm
zuschlag1543.htm
zuschlag1225.htm
zuschlag2738.htm
zuschlag1231.htm
zuschlag1557.htm
zuschlag2076.htm
zuschlag2710.htm
zuschlag1219.htm
zuschlag499.htm
zuschlag2704.htm
zuschlag2062.htm
zuschlag670.htm
zuschlag116.htm
zuschlag1782.htm
zuschlag102.htm
zuschlag1796.htm
zuschlag664.htm
zuschlag894.htm
zu

zuschlag2953.htm
zuschlag1282.htm
zuschlag416.htm
zuschlag370.htm
zuschlag364.htm
zuschlag1296.htm
zuschlag402.htm
zuschlag2947.htm
zuschlag358.htm
zuschlag1335.htm
zuschlag87.htm
zuschlag1453.htm
zuschlag1447.htm
zuschlag2628.htm
zuschlag1321.htm
zuschlag93.htm
zuschlag2600.htm
zuschlag1309.htm
zuschlag2166.htm
zuschlag2172.htm
zuschlag589.htm
zuschlag2614.htm
zuschlag2827.htm
zuschlag562.htm
zuschlag44.htm
zuschlag1490.htm
zuschlag204.htm
zuschlag2199.htm
zuschlag1484.htm
zuschlag210.htm
zuschlag50.htm
zuschlag576.htm
zuschlag2833.htm
zuschlag78.htm
zuschlag238.htm
zuschlag1651.htm
zuschlag1889.htm
zuschlag3046.htm
zuschlag2358.htm
zuschlag1137.htm
zuschlag1123.htm
zuschlag3052.htm
zuschlag1645.htm
zuschlag2364.htm
zuschlag947.htm
zuschlag2402.htm
zuschlag2416.htm
zuschlag953.htm
zuschlag1679.htm
zuschlag2370.htm
zuschlag3085.htm
zuschlag1692.htm
zuschlag760.htm
zuschlag774.htm
zuschlag1686.htm
zuschlag3091.htm
zuschlag1876.htm
zuschlag984.htm
zuschlag990.htm
zuschlag748.htm
zuschlag

zuschlag2406.htm
zuschlag943.htm
zuschlag1669.htm
zuschlag2360.htm
zuschlag2837.htm
zuschlag572.htm
zuschlag54.htm
zuschlag214.htm
zuschlag1480.htm
zuschlag2189.htm
zuschlag200.htm
zuschlag1494.htm
zuschlag40.htm
zuschlag566.htm
zuschlag2823.htm
zuschlag68.htm
zuschlag228.htm
zuschlag97.htm
zuschlag1325.htm
zuschlag1443.htm
zuschlag1457.htm
zuschlag2638.htm
zuschlag83.htm
zuschlag1331.htm
zuschlag2610.htm
zuschlag1319.htm
zuschlag2176.htm
zuschlag2162.htm
zuschlag599.htm
zuschlag2604.htm
zuschlag2943.htm
zuschlag406.htm
zuschlag1292.htm
zuschlag360.htm
zuschlag374.htm
zuschlag412.htm
zuschlag1286.htm
zuschlag2957.htm
zuschlag348.htm
zuschlag1251.htm
zuschlag2980.htm
zuschlag2758.htm
zuschlag1537.htm
zuschlag1523.htm
zuschlag2994.htm
zuschlag1245.htm
zuschlag2764.htm
zuschlag2002.htm
zuschlag2016.htm
zuschlag1279.htm
zuschlag2770.htm
zuschlag162.htm
zuschlag1090.htm
zuschlag604.htm
zuschlag2599.htm
zuschlag1084.htm
zuschlag610.htm
zuschlag176.htm
zuschlag1912.htm
zuschlag638.htm
zuschla

In [38]:
df = pd.DataFrame(list_df)
df["Zuschlag"].value_counts().head(10)

Daniel Fournier Agencement SA, Zweigniederlassung Regensdorf, Eichwatt 3,  8105  Regensdorf,  Schweiz                                                                                                                                                                                                                   91
WALAIR AG, Glattalstrasse 14,  8052  Zürich,  Schweiz                                                                                                                                                                                                                                                                   55
Bachmann &amp; Rimensberger AG, Freihofstrasse 20,  8700  Küsnacht,  Schweiz                                                                                                                                                                                                                                            54
Signal AG, Kanalstrasse 34,  3294  Büren an der Aare,  

### Klappt plus/minus
- evtl. Nur Name bis zum Komma nehmen. 
- Sonderfall Spahiu

### Dann also jetzt wieter zum Preis

In [71]:
#test2 im Loop (Zuschlag):
allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")
    datablock = datablock.replace("\'", "")   ### Das hier ist neu wegen Zahl!

    datablock
    
    preis = np.nan
    preisspanne_von = np.nan
    preisspanne_bis = np.nan
    
    if "CHF" in datablock:    #Checken, obs überhaupt eine Angabe drin hat
    
        try:
            regex3 = r"(?<=CHF)\s*\d*\.\d\d"
            preis = re.findall(regex3, datablock)[0]
        except:
            try: 
                regex3 = r"(?<=von)\s*\d*\.\d\d"
                regex4 = r"(?<=bis)\s*\d*\.\d\d"
                preisspanne_von = re.findall(regex3, datablock)[0].strip()
                preisspanne_bis = re.findall(regex4, datablock)[0].strip()
            except: 
                regex3 = r"(?<=de)\s*\d*\.\d\d"
                regex4 = r"(?<=à)\s*\d*\.\d\d"
                preisspanne_von = re.findall(regex3, datablock)[0].strip()
                preisspanne_bis = re.findall(regex4, datablock)[0].strip()  

    mini_dict = {"Preis" : preis, "Preisspanne von" : preisspanne_von, "Preisspanne bis" : preisspanne_bis}
    return mini_dict


for file in allefiles:
    if "zuschlag" in file: 
        data = open("simap1_2019/"+str(file), "r")
        text = data.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print(file)
        list_df.append(my_function(soup))### Probieren wirs also nun mit dem Preis

zuschlag768.htm
zuschlag2393.htm
zuschlag1842.htm
zuschlag1856.htm
zuschlag3099.htm
zuschlag2387.htm
zuschlag754.htm
zuschlag740.htm
zuschlag998.htm
zuschlag2436.htm
zuschlag973.htm
zuschlag1881.htm
zuschlag1659.htm
zuschlag2350.htm
zuschlag2344.htm
zuschlag1895.htm
zuschlag967.htm
zuschlag2422.htm
zuschlag1103.htm
zuschlag797.htm
zuschlag3072.htm
zuschlag1665.htm
zuschlag1671.htm
zuschlag2378.htm
zuschlag3066.htm
zuschlag1117.htm
zuschlag783.htm
zuschlag1498.htm
zuschlag2191.htm
zuschlag58.htm
zuschlag2185.htm
zuschlag218.htm
zuschlag230.htm
zuschlag556.htm
zuschlag70.htm
zuschlag2813.htm
zuschlag2807.htm
zuschlag64.htm
zuschlag542.htm
zuschlag224.htm
zuschlag2152.htm
zuschlag2634.htm
zuschlag2620.htm
zuschlag1329.htm
zuschlag2146.htm
zuschlag1467.htm
zuschlag2608.htm
zuschlag595.htm
zuschlag1301.htm
zuschlag581.htm
zuschlag1315.htm
zuschlag1473.htm
zuschlag378.htm
zuschlag2783.htm
zuschlag2797.htm
zuschlag344.htm
zuschlag422.htm
zuschlag2967.htm
zuschlag2973.htm
zuschlag436.htm
zusch

zuschlag829.htm
zuschlag1065.htm
zuschlag1071.htm
zuschlag2578.htm
zuschlag3100.htm
zuschlag1717.htm
zuschlag183.htm
zuschlag3128.htm
zuschlag2236.htm
zuschlag815.htm
zuschlag1059.htm
zuschlag2550.htm
zuschlag2544.htm
zuschlag801.htm
zuschlag2222.htm
zuschlag800.htm
zuschlag2545.htm
zuschlag2223.htm
zuschlag2237.htm
zuschlag3129.htm
zuschlag2551.htm
zuschlag1058.htm
zuschlag814.htm
zuschlag2579.htm
zuschlag1070.htm
zuschlag182.htm
zuschlag1716.htm
zuschlag3101.htm
zuschlag3115.htm
zuschlag196.htm
zuschlag1702.htm
zuschlag1064.htm
zuschlag828.htm
zuschlag2586.htm
zuschlag1931.htm
zuschlag169.htm
zuschlag1925.htm
zuschlag2592.htm
zuschlag627.htm
zuschlag141.htm
zuschlag155.htm
zuschlag1919.htm
zuschlag633.htm
zuschlag1528.htm
zuschlag2021.htm
zuschlag2747.htm
zuschlag2753.htm
zuschlag2035.htm
zuschlag1514.htm
zuschlag380.htm
zuschlag1272.htm
zuschlag1266.htm
zuschlag1500.htm
zuschlag394.htm
zuschlag2009.htm
zuschlag419.htm
zuschlag2784.htm
zuschlag2948.htm
zuschlag2790.htm
zuschlag1299.h

zuschlag3010.htm
zuschlag1607.htm
zuschlag1161.htm
zuschlag2468.htm
zuschlag1149.htm
zuschlag905.htm
zuschlag2440.htm
zuschlag3038.htm
zuschlag2326.htm
zuschlag2332.htm
zuschlag2454.htm
zuschlag911.htm
zuschlag722.htm
zuschlag1808.htm
zuschlag736.htm
zuschlag2483.htm
zuschlag1834.htm
zuschlag1820.htm
zuschlag2497.htm
zuschlag1836.htm
zuschlag1188.htm
zuschlag2481.htm
zuschlag2495.htm
zuschlag708.htm
zuschlag1822.htm
zuschlag720.htm
zuschlag734.htm
zuschlag2324.htm
zuschlag2442.htm
zuschlag907.htm
zuschlag913.htm
zuschlag2456.htm
zuschlag2330.htm
zuschlag1639.htm
zuschlag2318.htm
zuschlag3006.htm
zuschlag1611.htm
zuschlag1177.htm
zuschlag1163.htm
zuschlag1605.htm
zuschlag3012.htm
zuschlag2683.htm
zuschlag38.htm
zuschlag278.htm
zuschlag2697.htm
zuschlag522.htm
zuschlag2867.htm
zuschlag244.htm
zuschlag250.htm
zuschlag2873.htm
zuschlag10.htm
zuschlag536.htm
zuschlag1349.htm
zuschlag0.htm
zuschlag2640.htm
zuschlag2898.htm
zuschlag2126.htm
zuschlag2132.htm
zuschlag2654.htm
zuschlag1375.htm
z

zuschlag875.htm
zuschlag1039.htm
zuschlag2257.htm
zuschlag1986.htm
zuschlag1038.htm
zuschlag874.htm
zuschlag2531.htm
zuschlag2525.htm
zuschlag860.htm
zuschlag1992.htm
zuschlag2243.htm
zuschlag1762.htm
zuschlag1004.htm
zuschlag848.htm
zuschlag690.htm
zuschlag1010.htm
zuschlag684.htm
zuschlag2519.htm
zuschlag1776.htm
zuschlag109.htm
zuschlag1945.htm
zuschlag2294.htm
zuschlag2280.htm
zuschlag1789.htm
zuschlag1951.htm
zuschlag135.htm
zuschlag1979.htm
zuschlag653.htm
zuschlag647.htm
zuschlag121.htm
zuschlag2733.htm
zuschlag2055.htm
zuschlag2041.htm
zuschlag1548.htm
zuschlag2727.htm
zuschlag492.htm
zuschlag1206.htm
zuschlag2069.htm
zuschlag1560.htm
zuschlag1574.htm
zuschlag486.htm
zuschlag1212.htm
zuschlag2928.htm
zuschlag2096.htm
zuschlag2082.htm
zuschlag479.htm
zuschlag2914.htm
zuschlag451.htm
zuschlag337.htm
zuschlag323.htm
zuschlag445.htm
zuschlag2900.htm
zuschlag2647.htm
zuschlag7.htm
zuschlag1428.htm
zuschlag2121.htm
zuschlag2135.htm
zuschlag2653.htm
zuschlag1372.htm
zuschlag280.htm
zu

zuschlag2472.htm
zuschlag937.htm
zuschlag260.htm
zuschlag2843.htm
zuschlag1392.htm
zuschlag506.htm
zuschlag20.htm
zuschlag1386.htm
zuschlag34.htm
zuschlag512.htm
zuschlag2857.htm
zuschlag274.htm
zuschlag248.htm
zuschlag1437.htm
zuschlag1351.htm
zuschlag2658.htm
zuschlag2880.htm
zuschlag2894.htm
zuschlag1345.htm
zuschlag1423.htm
zuschlag2102.htm
zuschlag2664.htm
zuschlag1379.htm
zuschlag2670.htm
zuschlag2116.htm
zuschlag1580.htm
zuschlag314.htm
zuschlag2089.htm
zuschlag2937.htm
zuschlag472.htm
zuschlag466.htm
zuschlag2923.htm
zuschlag1594.htm
zuschlag300.htm
zuschlag328.htm
zuschlag1543.htm
zuschlag1225.htm
zuschlag2738.htm
zuschlag1231.htm
zuschlag1557.htm
zuschlag2076.htm
zuschlag2710.htm
zuschlag1219.htm
zuschlag499.htm
zuschlag2704.htm
zuschlag2062.htm
zuschlag670.htm
zuschlag116.htm
zuschlag1782.htm
zuschlag102.htm
zuschlag1796.htm
zuschlag664.htm
zuschlag894.htm
zuschlag1966.htm
zuschlag1972.htm
zuschlag880.htm
zuschlag658.htm
zuschlag1027.htm
zuschlag1741.htm
zuschlag1999.htm
zus

zuschlag2947.htm
zuschlag358.htm
zuschlag1335.htm
zuschlag87.htm
zuschlag1453.htm
zuschlag1447.htm
zuschlag2628.htm
zuschlag1321.htm
zuschlag93.htm
zuschlag2600.htm
zuschlag1309.htm
zuschlag2166.htm
zuschlag2172.htm
zuschlag589.htm
zuschlag2614.htm
zuschlag2827.htm
zuschlag562.htm
zuschlag44.htm
zuschlag1490.htm
zuschlag204.htm
zuschlag2199.htm
zuschlag1484.htm
zuschlag210.htm
zuschlag50.htm
zuschlag576.htm
zuschlag2833.htm
zuschlag78.htm
zuschlag238.htm
zuschlag1651.htm
zuschlag1889.htm
zuschlag3046.htm
zuschlag2358.htm
zuschlag1137.htm
zuschlag1123.htm
zuschlag3052.htm
zuschlag1645.htm
zuschlag2364.htm
zuschlag947.htm
zuschlag2402.htm
zuschlag2416.htm
zuschlag953.htm
zuschlag1679.htm
zuschlag2370.htm
zuschlag3085.htm
zuschlag1692.htm
zuschlag760.htm
zuschlag774.htm
zuschlag1686.htm
zuschlag3091.htm
zuschlag1876.htm
zuschlag984.htm
zuschlag990.htm
zuschlag748.htm
zuschlag1862.htm
zuschlag986.htm
zuschlag1874.htm
zuschlag1860.htm
zuschlag992.htm
zuschlag762.htm
zuschlag1690.htm
zuschla

zuschlag214.htm
zuschlag1480.htm
zuschlag2189.htm
zuschlag200.htm
zuschlag1494.htm
zuschlag40.htm
zuschlag566.htm
zuschlag2823.htm
zuschlag68.htm
zuschlag228.htm
zuschlag97.htm
zuschlag1325.htm
zuschlag1443.htm
zuschlag1457.htm
zuschlag2638.htm
zuschlag83.htm
zuschlag1331.htm
zuschlag2610.htm
zuschlag1319.htm
zuschlag2176.htm
zuschlag2162.htm
zuschlag599.htm
zuschlag2604.htm
zuschlag2943.htm
zuschlag406.htm
zuschlag1292.htm
zuschlag360.htm
zuschlag374.htm
zuschlag412.htm
zuschlag1286.htm
zuschlag2957.htm
zuschlag348.htm
zuschlag1251.htm
zuschlag2980.htm
zuschlag2758.htm
zuschlag1537.htm
zuschlag1523.htm
zuschlag2994.htm
zuschlag1245.htm
zuschlag2764.htm
zuschlag2002.htm
zuschlag2016.htm
zuschlag1279.htm
zuschlag2770.htm
zuschlag162.htm
zuschlag1090.htm
zuschlag604.htm
zuschlag2599.htm
zuschlag1084.htm
zuschlag610.htm
zuschlag176.htm
zuschlag1912.htm
zuschlag638.htm
zuschlag1906.htm
zuschlag1735.htm
zuschlag3122.htm
zuschlag1053.htm
zuschlag1047.htm
zuschlag3136.htm
zuschlag2228.htm
zus

In [73]:
df = pd.DataFrame(list_df)
df.head(15)

,Preis,Preisspanne von,Preisspanne bis
0,870642.75,NaN,NaN
1,140901.00,NaN,NaN
2,638149.05,NaN,NaN
3,145000.00,NaN,NaN
4,915597.00,NaN,NaN
5,244620.00,NaN,NaN
6,1760895.00,NaN,NaN
7,159430.00,NaN,NaN
8,68488.60,NaN,NaN
9,462525.80,NaN,NaN


### Super, und jetzt das Projekt

'<td><h1>Zuschlag</h1><dl><dd><span class="label">Publikationsdatum Simap:</span> 26.06.2019<br/></dd></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span> Stadt Wetzikon<br/><span class="label">Beschaffungsstelle/Organisator:</span> Stadt Wetzikon<br/>Abteilung Tiefbau, Bahnhofstrasse 167,  8622  Wetzikon,  Schweiz,    Telefon:    044 931 32 87,  E-Mail:  <a href="mailto:claude.zbinden@wetzikon.ch">claude.zbinden@wetzikon.ch</a><br/></dd></dl><h3>1.2 Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3 Verfahrensart </h3><dl><dd> Offenes Verfahren <br/></dd></dl><h3>1.4 Auftragsart </h3><dl><dd> Bauauftrag <br/></dd></dl><h3>1.5 Gemäss GATT/WTO-Abkommen, resp. Staatsvertrag </h3><dl><dd> Nein <br/></dd></dl><h2>2. Beschaffungsobjekt</h2><h3>2.1 Projekttitel der Beschaffung</h3><dl><dd>Erneuerung Hofstrasse</dd></dl><h3>2.2 Gemeinschaftsvokabular</h3><dl><dd><table><tbody><tr>

In [79]:
#test2 im Loop (Zuschlag):
allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")
    datablock = datablock.replace("\'", "")   ### Das hier ist neu wegen Zahl!

    datablock
        
    try:
        regex5 = r"(?<=2\.1 Projekttitel der Beschaffung<\/h3><dl><dd>).*(?=<\/dd><\/dl><h3>2\.2)"
        projekt = re.findall(regex5, datablock)[0]
    except:
        regex5 = r"(?<=2\.1 Titre du projet du marché<\/h3><dl><dd>).*(?=<\/dd><\/dl><h3>2\.2)"
        projekt = re.findall(regex5, datablock)[0]

    mini_dict = {"Projekt" : projekt}
    return mini_dict


for file in allefiles:
    if "zuschlag" in file: 
        data = open("simap1_2019/"+str(file), "r")
        text = data.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print(file)
        list_df.append(my_function(soup))

zuschlag768.htm
zuschlag2393.htm
zuschlag1842.htm
zuschlag1856.htm
zuschlag3099.htm
zuschlag2387.htm
zuschlag754.htm
zuschlag740.htm
zuschlag998.htm
zuschlag2436.htm
zuschlag973.htm
zuschlag1881.htm
zuschlag1659.htm
zuschlag2350.htm
zuschlag2344.htm
zuschlag1895.htm
zuschlag967.htm
zuschlag2422.htm
zuschlag1103.htm
zuschlag797.htm
zuschlag3072.htm
zuschlag1665.htm
zuschlag1671.htm
zuschlag2378.htm
zuschlag3066.htm
zuschlag1117.htm
zuschlag783.htm
zuschlag1498.htm
zuschlag2191.htm
zuschlag58.htm
zuschlag2185.htm
zuschlag218.htm
zuschlag230.htm
zuschlag556.htm
zuschlag70.htm
zuschlag2813.htm
zuschlag2807.htm
zuschlag64.htm
zuschlag542.htm
zuschlag224.htm
zuschlag2152.htm
zuschlag2634.htm
zuschlag2620.htm
zuschlag1329.htm
zuschlag2146.htm
zuschlag1467.htm
zuschlag2608.htm
zuschlag595.htm
zuschlag1301.htm
zuschlag581.htm
zuschlag1315.htm
zuschlag1473.htm
zuschlag378.htm
zuschlag2783.htm
zuschlag2797.htm
zuschlag344.htm
zuschlag422.htm
zuschlag2967.htm
zuschlag2973.htm
zuschlag436.htm
zusch

zuschlag1930.htm
zuschlag1703.htm
zuschlag197.htm
zuschlag3114.htm
zuschlag829.htm
zuschlag1065.htm
zuschlag1071.htm
zuschlag2578.htm
zuschlag3100.htm
zuschlag1717.htm
zuschlag183.htm
zuschlag3128.htm
zuschlag2236.htm
zuschlag815.htm
zuschlag1059.htm
zuschlag2550.htm
zuschlag2544.htm
zuschlag801.htm
zuschlag2222.htm
zuschlag800.htm
zuschlag2545.htm
zuschlag2223.htm
zuschlag2237.htm
zuschlag3129.htm
zuschlag2551.htm
zuschlag1058.htm
zuschlag814.htm
zuschlag2579.htm
zuschlag1070.htm
zuschlag182.htm
zuschlag1716.htm
zuschlag3101.htm
zuschlag3115.htm
zuschlag196.htm
zuschlag1702.htm
zuschlag1064.htm
zuschlag828.htm
zuschlag2586.htm
zuschlag1931.htm
zuschlag169.htm
zuschlag1925.htm
zuschlag2592.htm
zuschlag627.htm
zuschlag141.htm
zuschlag155.htm
zuschlag1919.htm
zuschlag633.htm
zuschlag1528.htm
zuschlag2021.htm
zuschlag2747.htm
zuschlag2753.htm
zuschlag2035.htm
zuschlag1514.htm
zuschlag380.htm
zuschlag1272.htm
zuschlag1266.htm
zuschlag1500.htm
zuschlag394.htm
zuschlag2009.htm
zuschlag419.ht

zuschlag939.htm
zuschlag1613.htm
zuschlag3004.htm
zuschlag3010.htm
zuschlag1607.htm
zuschlag1161.htm
zuschlag2468.htm
zuschlag1149.htm
zuschlag905.htm
zuschlag2440.htm
zuschlag3038.htm
zuschlag2326.htm
zuschlag2332.htm
zuschlag2454.htm
zuschlag911.htm
zuschlag722.htm
zuschlag1808.htm
zuschlag736.htm
zuschlag2483.htm
zuschlag1834.htm
zuschlag1820.htm
zuschlag2497.htm
zuschlag1836.htm
zuschlag1188.htm
zuschlag2481.htm
zuschlag2495.htm
zuschlag708.htm
zuschlag1822.htm
zuschlag720.htm
zuschlag734.htm
zuschlag2324.htm
zuschlag2442.htm
zuschlag907.htm
zuschlag913.htm
zuschlag2456.htm
zuschlag2330.htm
zuschlag1639.htm
zuschlag2318.htm
zuschlag3006.htm
zuschlag1611.htm
zuschlag1177.htm
zuschlag1163.htm
zuschlag1605.htm
zuschlag3012.htm
zuschlag2683.htm
zuschlag38.htm
zuschlag278.htm
zuschlag2697.htm
zuschlag522.htm
zuschlag2867.htm
zuschlag244.htm
zuschlag250.htm
zuschlag2873.htm
zuschlag10.htm
zuschlag536.htm
zuschlag1349.htm
zuschlag0.htm
zuschlag2640.htm
zuschlag2898.htm
zuschlag2126.htm
zu

zuschlag1993.htm
zuschlag1987.htm
zuschlag2256.htm
zuschlag2530.htm
zuschlag875.htm
zuschlag1039.htm
zuschlag2257.htm
zuschlag1986.htm
zuschlag1038.htm
zuschlag874.htm
zuschlag2531.htm
zuschlag2525.htm
zuschlag860.htm
zuschlag1992.htm
zuschlag2243.htm
zuschlag1762.htm
zuschlag1004.htm
zuschlag848.htm
zuschlag690.htm
zuschlag1010.htm
zuschlag684.htm
zuschlag2519.htm
zuschlag1776.htm
zuschlag109.htm
zuschlag1945.htm
zuschlag2294.htm
zuschlag2280.htm
zuschlag1789.htm
zuschlag1951.htm
zuschlag135.htm
zuschlag1979.htm
zuschlag653.htm
zuschlag647.htm
zuschlag121.htm
zuschlag2733.htm
zuschlag2055.htm
zuschlag2041.htm
zuschlag1548.htm
zuschlag2727.htm
zuschlag492.htm
zuschlag1206.htm
zuschlag2069.htm
zuschlag1560.htm
zuschlag1574.htm
zuschlag486.htm
zuschlag1212.htm
zuschlag2928.htm
zuschlag2096.htm
zuschlag2082.htm
zuschlag479.htm
zuschlag2914.htm
zuschlag451.htm
zuschlag337.htm
zuschlag323.htm
zuschlag445.htm
zuschlag2900.htm
zuschlag2647.htm
zuschlag7.htm
zuschlag1428.htm
zuschlag2121.htm
z

zuschlag1621.htm
zuschlag1147.htm
zuschlag923.htm
zuschlag2466.htm
zuschlag2300.htm
zuschlag1609.htm
zuschlag2314.htm
zuschlag2472.htm
zuschlag937.htm
zuschlag260.htm
zuschlag2843.htm
zuschlag1392.htm
zuschlag506.htm
zuschlag20.htm
zuschlag1386.htm
zuschlag34.htm
zuschlag512.htm
zuschlag2857.htm
zuschlag274.htm
zuschlag248.htm
zuschlag1437.htm
zuschlag1351.htm
zuschlag2658.htm
zuschlag2880.htm
zuschlag2894.htm
zuschlag1345.htm
zuschlag1423.htm
zuschlag2102.htm
zuschlag2664.htm
zuschlag1379.htm
zuschlag2670.htm
zuschlag2116.htm
zuschlag1580.htm
zuschlag314.htm
zuschlag2089.htm
zuschlag2937.htm
zuschlag472.htm
zuschlag466.htm
zuschlag2923.htm
zuschlag1594.htm
zuschlag300.htm
zuschlag328.htm
zuschlag1543.htm
zuschlag1225.htm
zuschlag2738.htm
zuschlag1231.htm
zuschlag1557.htm
zuschlag2076.htm
zuschlag2710.htm
zuschlag1219.htm
zuschlag499.htm
zuschlag2704.htm
zuschlag2062.htm
zuschlag670.htm
zuschlag116.htm
zuschlag1782.htm
zuschlag102.htm
zuschlag1796.htm
zuschlag664.htm
zuschlag894.htm
zu

zuschlag2006.htm
zuschlag1269.htm
zuschlag2760.htm
zuschlag2953.htm
zuschlag1282.htm
zuschlag416.htm
zuschlag370.htm
zuschlag364.htm
zuschlag1296.htm
zuschlag402.htm
zuschlag2947.htm
zuschlag358.htm
zuschlag1335.htm
zuschlag87.htm
zuschlag1453.htm
zuschlag1447.htm
zuschlag2628.htm
zuschlag1321.htm
zuschlag93.htm
zuschlag2600.htm
zuschlag1309.htm
zuschlag2166.htm
zuschlag2172.htm
zuschlag589.htm
zuschlag2614.htm
zuschlag2827.htm
zuschlag562.htm
zuschlag44.htm
zuschlag1490.htm
zuschlag204.htm
zuschlag2199.htm
zuschlag1484.htm
zuschlag210.htm
zuschlag50.htm
zuschlag576.htm
zuschlag2833.htm
zuschlag78.htm
zuschlag238.htm
zuschlag1651.htm
zuschlag1889.htm
zuschlag3046.htm
zuschlag2358.htm
zuschlag1137.htm
zuschlag1123.htm
zuschlag3052.htm
zuschlag1645.htm
zuschlag2364.htm
zuschlag947.htm
zuschlag2402.htm
zuschlag2416.htm
zuschlag953.htm
zuschlag1679.htm
zuschlag2370.htm
zuschlag3085.htm
zuschlag1692.htm
zuschlag760.htm
zuschlag774.htm
zuschlag1686.htm
zuschlag3091.htm
zuschlag1876.htm
zusch

zuschlag3042.htm
zuschlag1655.htm
zuschlag2374.htm
zuschlag957.htm
zuschlag2412.htm
zuschlag2406.htm
zuschlag943.htm
zuschlag1669.htm
zuschlag2360.htm
zuschlag2837.htm
zuschlag572.htm
zuschlag54.htm
zuschlag214.htm
zuschlag1480.htm
zuschlag2189.htm
zuschlag200.htm
zuschlag1494.htm
zuschlag40.htm
zuschlag566.htm
zuschlag2823.htm
zuschlag68.htm
zuschlag228.htm
zuschlag97.htm
zuschlag1325.htm
zuschlag1443.htm
zuschlag1457.htm
zuschlag2638.htm
zuschlag83.htm
zuschlag1331.htm
zuschlag2610.htm
zuschlag1319.htm
zuschlag2176.htm
zuschlag2162.htm
zuschlag599.htm
zuschlag2604.htm
zuschlag2943.htm
zuschlag406.htm
zuschlag1292.htm
zuschlag360.htm
zuschlag374.htm
zuschlag412.htm
zuschlag1286.htm
zuschlag2957.htm
zuschlag348.htm
zuschlag1251.htm
zuschlag2980.htm
zuschlag2758.htm
zuschlag1537.htm
zuschlag1523.htm
zuschlag2994.htm
zuschlag1245.htm
zuschlag2764.htm
zuschlag2002.htm
zuschlag2016.htm
zuschlag1279.htm
zuschlag2770.htm
zuschlag162.htm
zuschlag1090.htm
zuschlag604.htm
zuschlag2599.htm
zusch

In [80]:
df = pd.DataFrame(list_df)
df.head(15)

,Projekt
0,BAV 80451 Schulanlage Hofacker</dd><dd><b> Bes...
1,BKP 273.0 Innentüren aus Holz (Stahlzargentüren)
2,"Mattenbachstrasse, Zeughausstrasse bis Unterer..."
3,"Neubau Watterstrasse 117/121, 8105 Regensdorf"
4,"Stadtspital Triemli, Neubau Bettenhaus"
5,BAV 60459 Förrlibuckstrasse
6,BAV 80451 Schulanlage Hofacker</dd><dd><b> Bes...
7,BAV 40150 Pflegezentrum Bombach Haus B</dd><dd...
8,"Kongresshaus und Tonhalle Zürich, Gebäudeautom..."
9,Tiefbau und Belagsarbeiten Tunnelstrasse


In [77]:
# Das hier ist der Textblock, um Regexen zu prüfen, wenn nicht klappt...

file = open("simap1_2019/zuschlag2344.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

datablock = str(soup.find_all("td")[0])

datablock = datablock.replace("\n", " ")
datablock = datablock.replace("\xa0", " ")
datablock = datablock.replace("\n", "")
datablock = datablock.replace("\t", "")
datablock = datablock.replace("\'", "")

datablock

'<td><h1>Zuschlag</h1><dl><dd></dd><dd><b>Publikationsdatum Kantonales Amtsblatt</b></dd>   ZG  03.10.2014, ZH  03.10.2014<br/><span class="label">Publikationsdatum Simap:</span> 03.10.2014<br/></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span> AWEL Amt für Abfall, Wasser, Energie und Luft, Abt. Wasserbau<br/><span class="label">Beschaffungsstelle/Organisator:</span> AWEL Amt für Abfall, Wasser, Energie und Luft<br/>Abt. Wasserbau, Walcheplatz 2, Postfach,  8090  Zürich,  Schweiz,    Telefon:    043 259 32 24<br/></dd></dl><h3>1.2 Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3 Verfahrensart </h3><dl><dd> Offenes Verfahren <br/></dd></dl><h3>1.4 Auftragsart </h3><dl><dd> Bauauftrag <br/></dd></dl><h3>1.5 Gemäss GATT/WTO-Abkommen, resp. Staatsvertrag </h3><dl><dd> Nein <br/></dd></dl><h2>2. Beschaffungsobjekt</h2><h3>2.1 Projekttitel der Beschaffung</h3><dl><dd>HWS und 

## So, und jetzt hängen wir alles zusammen!

In [165]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import numpy as np

allefiles = os.listdir("simap1_2019")
list_df = []

def my_function(input):
    datablock = str(soup.find_all("td")[0])
    datablock = datablock.replace("\n", " ")
    datablock = datablock.replace("\xa0", " ")
    datablock = datablock.replace("\n", "")
    datablock = datablock.replace("\t", "")
    datablock = datablock.replace("\'", "")   ### Das hier ist neu wegen Zahl!
    datablock = datablock.replace("&amp;", "&") 

    datablock
    
    #PUBLIKATIONSDATUM
    try: 
        regex0 = r"(?<=Publikationsdatum Simap:<\/span>)\s*\d\d.\d\d.\d\d\d\d"
        datum = re.findall(regex0, datablock)[0].strip()
    except: 
        regex0 = r"(?<=publication Simap:<\/span>)\s*\d\d.\d\d.\d\d\d\d"
        datum = re.findall(regex0, datablock)[0].strip()
    
    
    #AUFTRAGGEBER
    try: 
        regex1 = r"(?<=Bedarfsstelle\/Vergabestelle:<\/span> ).*(?=<br\/><span class=\"label\">Beschaffungsstelle\/Organisator:)"
        auftraggeber = re.findall(regex1, datablock)[0]
    except: 
        regex1 = r"(?<=adjudicatrice:<\/span> ).*(?=<br\/><span class=\"label\">Service organisateur)"
        auftraggeber = re.findall(regex1, datablock)[0]
        
    auftraggeber_liste = auftraggeber.split(",")
    auftraggeber_kurz = auftraggeber_liste[0].strip()


    #ZUSCHLAG
    try: 
        regex2 = r"(?<=Name:<\/span> ).*(?=<br\/><span class=\"label\"\>Preis)"
        zuschlag = re.findall(regex2, datablock)[0]
    except:
        try: 
            regex2 = r"(?<=Nom:<\/span> ).*(?=<br\/><span class=\"label\"\>Prix)"
            zuschlag = re.findall(regex2, datablock)[0]
        except: 
            regex2 = r"(?<=Nom:<\/span> ).*(?=<br\/><span class=\"label\"\>Fourchette)"
            zuschlag = re.findall(regex2, datablock)[0]

    zuschlag_liste = zuschlag.split(",")
    zuschlag_kurz = zuschlag_liste[0].strip()
            
    #PREIS
    preis = np.nan
    preisspanne_von = np.nan
    preisspanne_bis = np.nan
    
    if "CHF" in datablock:    #Checken, obs überhaupt eine Angabe drin hat
    
        try:
            regex3 = r"(?<=CHF)\s*\d*\.\d\d"
            preis = re.findall(regex3, datablock)[0]
        except:
            try: 
                regex3 = r"(?<=von)\s*\d*\.\d\d"
                regex4 = r"(?<=bis)\s*\d*\.\d\d"
                preisspanne_von = re.findall(regex3, datablock)[0].strip()
                preisspanne_bis = re.findall(regex4, datablock)[0].strip()
            except: 
                regex3 = r"(?<=de)\s*\d*\.\d\d"
                regex4 = r"(?<=à)\s*\d*\.\d\d"
                preisspanne_von = re.findall(regex3, datablock)[0].strip()
                preisspanne_bis = re.findall(regex4, datablock)[0].strip()
                
                
    #PROJEKT
    try:
        regex5 = r"(?<=2\.1 Projekttitel der Beschaffung<\/h3><dl><dd>).*(?=<\/dd><\/dl><h3>2\.2)"
        projekt = re.findall(regex5, datablock)[0]
    except:
        regex5 = r"(?<=2\.1 Titre du projet du marché<\/h3><dl><dd>).*(?=<\/dd><\/dl><h3>2\.2)"
        projekt = re.findall(regex5, datablock)[0]
        

    mini_dict = {"Datum" : datum,
                 "Auftraggeber kurz" : auftraggeber_kurz,
                 "Auftraggeber" : auftraggeber,
                 "Zuschlag kurz" : zuschlag_kurz,
                 "Zuschlag" : zuschlag, 
                 "Preis" : preis, 
                 "Preisspanne von" : preisspanne_von, 
                 "Preisspanne bis" : preisspanne_bis, 
                 "Projekt" : projekt}
    
    return mini_dict

#LOOP

for file in allefiles:
    if "zuschlag" in file: 
        data = open("simap1_2019/"+str(file), "r")
        text = data.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #print(file)
        list_df.append(my_function(soup))


In [178]:
from datetime import datetime

df = pd.DataFrame(list_df)

#Umwandeln der Werte in Zahlen:

df["Preis"] = df["Preis"].astype(float)
df["Preisspanne von"] = df["Preisspanne von"].astype(float)
df["Preisspanne bis"] = df["Preisspanne bis"].astype(float)

df["Datum"] = pd.to_datetime(df['Datum'], format="%d.%m.%Y")
df = df.set_index("Datum")

df.sort_index(ascending = False).head(10)

,Auftraggeber kurz,Auftraggeber,Zuschlag kurz,Zuschlag,Preis,Preisspanne von,Preisspanne bis,Projekt
Datum,,,,,,,,
2019-10-25,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",Rota AG,"Rota AG, Rütiwisstrasse 3, 8820 Wädenswil, ...",72810.90,NaN,NaN,BAV 80451 Schulanlage Hofacker
2019-10-25,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",Gadola Bau AG,"Gadola Bau AG, Zweigniederlassung Uster, Gross...",202050.15,NaN,NaN,BAV 27171 Restaurant Fischerstube</dd><dd><b> ...
2019-10-22,Stadt Zürich,"Stadt Zürich, ERZ Entsorgung + Recycling Züric...",Specogna Bau AG,"Specogna Bau AG, Steinackerstrasse 55, 8302 ...",1549799.00,NaN,NaN,Anschlussbauwerk Expansionsbauwerk Hagenholz B...
2019-10-21,Politische Gemeinde,Politische Gemeinde,Egli Zimmerei AG,"Egli Zimmerei AG, Ausserdorfstrasse 2, 9621 ...",664180.60,NaN,NaN,Erweiterung Schulanlage Töss
2019-10-18,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",Dämmtech Swiss AG,"Dämmtech Swiss AG, Kirchleerauerstrasse 1, 50...",1054012.20,NaN,NaN,"BAV 41503 Stadtspital Triemli, Instandhaltung ..."
2019-10-18,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",Marti AG,"Marti AG, Bauunternehmung, Thurgauerstrasse 68...",2533940.95,NaN,NaN,"BAV 41503 Stadtspital Triemli, Instandhaltung ..."
2019-10-11,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",Walo Bertschinger AG,"Walo Bertschinger AG, Giessenstrasse 5, 8953 ...",542482.30,NaN,NaN,BAV 80451 Schulanlage Hofacker</dd><dd><b> Bes...
2019-10-11,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",KOCH Group AG Wallisellen,"KOCH Group AG Wallisellen, Birgistrasse 3, 83...",324127.55,NaN,NaN,BAV 40153 Alterszentrum Mathysweg</dd><dd><b> ...
2019-10-11,Stadt Zürich,"Stadt Zürich, Amt für Hochbauten",RYMA AG,"RYMA AG, Kirchstrasse 11, 8807 Freienbach, ...",147293.20,NaN,NaN,BAV 80451 Schulanlage Hofacker</dd><dd><b> Bes...


In [179]:
df["Zuschlag kurz"].value_counts()

Daniel Fournier Agencement SA    93
Signal AG                        55
WALAIR AG                        55
Bachmann & Rimensberger AG       54
Rowiss AG                        51
                                 ..
Meier Gartenbau AG                1
Streuli Bau AG                    1
ISS Kanal Services AG             1
Energie 360 Grad Erdwärme AG      1
ALPINE-ENERGIE Schweiz AG         1
Name: Zuschlag kurz, Length: 996, dtype: int64

In [180]:
# Das sind die besten Verdiener an Gemeinden:
df.groupby("Zuschlag")["Preis"].sum().round(2).sort_values(ascending = False)

Zuschlag
Blumer-Lehmann AG, Erlenhof,  9200  Gossau,  Schweiz                        1.355137e+08
Walo Bertschinger AG, Postfach 1155,  8021  Zürich,  Schweiz                9.035923e+07
Bachmann & Rimensberger AG, Freihofstrasse 20,  8700  Küsnacht,  Schweiz    6.353463e+07
REZZONICO IMPIANTI SA, Via Stazione 3,  6934  Bioggio,  Schweiz             6.137464e+07
Kummler + Matter AG, Hohlstrasse 176,  8004  Zürich,  Schweiz               5.952099e+07
                                                                                ...     
Vögeli Holzbau AG, Böttsteinerstrasse 2,  5314  Kleindöttingen,  Schweiz    0.000000e+00
ALPINE-ENERGIE Schweiz AG, Bifang 18,  4665  Oftringen,  Schweiz            0.000000e+00
W. Schmid AG,  8152  Glattbrugg,  Schweiz                                   0.000000e+00
Dollé Gartenbau GmbH, Erlengutstrasse 8,  8703  Erlenbach,  Schweiz         0.000000e+00
Arge wsb, Im Hard 8,  8197  Rafz,  Schweiz                                  0.000000e+00
Name: Preis,

In [181]:
df_new = df.groupby("Auftraggeber kurz")["Preis"].sum().round(2).sort_values(ascending = False)
pd.DataFrame(df_new).head(30)

,Preis
Auftraggeber kurz,
Stadt Zürich,1.347195e+09
Stadt Zürich Verkehrsbetriebe Zürich,1.736894e+08
Stadt Winterthur Departement Bau,1.185770e+08
Stadt Winterthur Amt für Städtebau,9.505463e+07
Gemeinde Meilen,9.287218e+07
Stadt Uster,8.806198e+07
ewz Elektrizitätswerk der Stadt Zürich,7.852696e+07
Einfache Gesellschaft Kunsthaus Erweiterung,5.937362e+07
Stadt Zürich ERZ Entsorgung + Recycling Zürich,4.870492e+07


In [183]:
df[df["Auftraggeber"] == "Stadt Opfikon Glattbrugg"]

,Auftraggeber kurz,Auftraggeber,Zuschlag kurz,Zuschlag,Preis,Preisspanne von,Preisspanne bis,Projekt
Datum,,,,,,,,
2019-04-18,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Hüsser Innenausbau AG,"Hüsser Innenausbau AG, Oberebenestrasse 20, 5...",494260.30,NaN,NaN,Innentüren aus Holz
2019-04-18,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Gawo Gasser AG,"Gawo Gasser AG, Entlebucherstrasse 46, 6110 ...",331130.85,NaN,NaN,Fenster in Holz-Metall
2018-10-30,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Egli AG Gips- und Fassadensysteme,"Egli AG Gips- und Fassadensysteme, Längfeldweg...",996749.65,NaN,NaN,Gipserarbeiten
2016-09-30,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Karl Waechter AG,"Karl Waechter AG, Forchstrasse 45, 8032 Züri...",395000.00,NaN,NaN,Sanitäranlagen
2018-07-20,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Bühlmann AG,"Bühlmann AG, Giessenstrasse 11 B, 8953 Dieti...",262246.55,NaN,NaN,Montagebau in Holz
2016-09-30,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Melcom AG,"Melcom AG, Hertistrasse 25, 8304 Wallisellen...",1039161.55,NaN,NaN,Elektroanlagen
2019-04-18,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Creatop AG,"Creatop AG, Etzelstrasse 7, 8730 Uznach, Sc...",707387.35,NaN,NaN,Allgemeine Schreinerarbeiten
2016-09-30,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Ammann + Schmid AG,"Ammann + Schmid AG, Heizungsinstallationen + w...",401694.55,NaN,NaN,Heizungsanlagen
2016-09-30,Stadt Opfikon Glattbrugg,Stadt Opfikon Glattbrugg,Airba AG,"Airba AG, Rümikerstrasse 14, 8409 Winterthur...",776812.70,NaN,NaN,Lüftungsanlagen


In [184]:
meiste_aufträge = df["Auftraggeber"].value_counts()
df_aufträge = pd.DataFrame(meiste_aufträge)
df_aufträge.head(20)

,Auftraggeber
Stadt Zürich,989
"Stadt Zürich, Amt für Hochbauten",282
"Gemeinde Meilen, Liegenschaftsabteilung",150
Stadt Winterthur Amt für Städtebau,100
ewz Elektrizitätswerk der Stadt Zürich,86
"Stadt Zürich, Dienstabteilung Verkehr",74
"Stadt Uster, Abteilung Gesundheit",63
Stadt Winterthur Departement Bau,52
Zürichsee Schifffahrtsgesellschaft,50
Politische Gemeinde Erlenbach Liegenschaftenkommission,42
